# Langchain for GenAI

> Playlist: [CampusX](https://www.youtube.com/playlist?list=PLKnIA16_RmvaTbihpo4MtzVm4XOQa0ER0)

> Notes: https://web.goodnotes.com/s/zBlfqECThNwAJhid4A8bGe

---

## Testing out the environment

In [67]:
import langchain
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from dotenv import load_dotenv
from pprint import pprint
import random
from loguru import logger

In [2]:
load_dotenv()  # Load environment variables from a .env file if present

True

In [3]:
langchain.__version__

'0.3.27'

In [4]:
gemini_flash_model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [5]:
model = ChatGroq(
    model="openai/gpt-oss-20b",
    temperature=0,
    max_tokens=None,
    reasoning_format="parsed",
    timeout=None,
    max_retries=2,
)

In [6]:
model.invoke("what is the capital of india").content

'The capital of India is **New\u202fDelhi**.'

In [7]:
gemini_flash_model.invoke("what is the capital of india").content

'The capital of India is **New Delhi**.'

### Embedding model

In [ ]:
embedding_model = GoogleGenerativeAIEmbeddings(model="gemini-embedding-001")
embeddings = embedding_model.embed_query(text="What's our Q1 revenue?", output_dimensionality=10)

[-0.03572908416390419,
 0.014558478258550167,
 0.011592254973948002,
 -0.08969993889331818,
 -0.009068180806934834,
 0.013664662837982178,
 0.011340967379510403,
 -0.005701108369976282,
 -0.027033332735300064,
 3.775993536692113e-05]

In [16]:
from langchain_huggingface import HuggingFaceEmbeddings

In [19]:
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={"device": "cpu"},
)
embeddings = embedding_model.embed_query(text="What's our Q1 revenue?")

In [22]:
len(embeddings)

384

## Langchain Prompts

### Basic Prompt template

In [8]:
from langchain_core.prompts import PromptTemplate, load_prompt

In [62]:
template = PromptTemplate(
    template="""
    You are a helpful assistant that can generate a short report on the topic: {paper_input}
    The report should be in the style of {style_input} and the length should be {length_input}
    """,
    input_variables=["paper_input", "style_input", "length_input"]
)

template.save("./prompt_templates/template.json")

In [63]:
template = load_prompt("./prompt_templates/template.json")

In [64]:
prompt = template.format(
        paper_input="Attention is all you need", style_input="Beginner-Friendly", length_input="Short (1-2 paragraphs)"
    )

In [67]:
print (prompt)

### Messages


    You are a helpful assistant that can generate a short report on the topic: Attention is all you need
    The report should be in the style of Beginner-Friendly and the length should be Short (1-2 paragraphs)
    


### Messages

In [7]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [35]:
messages = [
    SystemMessage(content="You are a helpful assistant that can answer questions and help with tasks."),
    HumanMessage(content="What is the capital of France?"),
]

result = model.invoke(messages)

result.content

'The capital of France is **Paris**.'

### Dynammic list of messages

In [39]:
chat_template = ChatPromptTemplate(messages=[
    SystemMessage(content="You are a helpful assistant who is an expert in the domain: {domain}"),
    HumanMessage(content="Explain the topic in simple terms: {topic}"),
])

In [42]:
prompt = chat_template.invoke({"domain": "AI", "topic": "Self Attention"})

print(prompt)

messages=[SystemMessage(content='You are a helpful assistant who is an expert in the domain: {domain}', additional_kwargs={}, response_metadata={}), HumanMessage(content='Explain the topic in simple terms: {topic}', additional_kwargs={}, response_metadata={})]


### The above does not work

In [47]:
chat_template = ChatPromptTemplate(messages=[
    ("system", "You are a helpful assistant who is an expert in the domain: {domain}"),
    ("human", "Explain the topic in simple terms in 3-5 sentences: {topic}"),
])
prompt = chat_template.invoke({"domain": "AI", "topic": "Self Attention"})

print(prompt)

messages=[SystemMessage(content='You are a helpful assistant who is an expert in the domain: AI', additional_kwargs={}, response_metadata={}), HumanMessage(content='Explain the topic in simple terms in 3-5 sentences: Self Attention', additional_kwargs={}, response_metadata={})]


In [48]:
result = model.invoke(prompt)
print(result.content)


Self‑attention is a way for a model to look at all the words in a sentence at once and decide how much each word should influence every other word.  
For each word, the model creates three vectors—query, key, and value—then compares the query of one word with the keys of all words to get a “similarity score.”  
These scores are turned into weights (via a softmax) that say how much attention each word should give to the others, and the weighted sum of the value vectors gives the new representation for that word.  
Because every word can attend to every other word, the model captures long‑range relationships and context without needing to process the sentence sequentially.  
This mechanism is the core of transformer models, enabling them to understand and generate language efficiently.


### Message Placeholder

In [72]:
history_messages = [
    HumanMessage(content="I want to request a refund for my order #12345."),
    AIMessage(content="Your refund request for order #12345 has been initiated. It will be processed in 3-5 business days.")
]

In [75]:
history_messages[0].content


'I want to request a refund for my order #12345.'

In [71]:
chat_template = ChatPromptTemplate(messages=[
    ("system", "You are a helpful assistant that can answer questions and help with tasks."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{query}"),
])

prompt = chat_template.invoke({"query": "how many day again?", "chat_history": history_messages})

pprint(prompt)

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant that can answer questions and help with tasks.', additional_kwargs={}, response_metadata={}), HumanMessage(content='I want to request a refund for my order #12345.', additional_kwargs={}, response_metadata={}), AIMessage(content='Your refund request for order #12345 has been initiated. It will be processed in 3-5 business days.', additional_kwargs={}, response_metadata={}), HumanMessage(content='how many day again?', additional_kwargs={}, response_metadata={})])


In [57]:
response = model.invoke(prompt)
print(response.content)

It will take **3–5 business days** to complete the refund.  
That means the processing time is counted only on weekdays (Monday‑Friday), excluding public holidays. If you have any more questions, just let me know!


## Structured Outputs

### Typed Dict

In [20]:
from typing import TypedDict, Annotated, Optional, Literal, List

class Person(TypedDict):
    name: str
    age: int
    email: str

person = Person(name="John", age=30, email="john@example.com")

In [ ]:
person = Person(name=123, age=30, email="john@example.com") # this is wrong according to the type def, but it will not raise an error

In [78]:
# create a dummy review for a mobile phone in plain text
review = """
This is a decent phone! I love the camera and the battery life is amazing. Also, the price is reasonable. The display is ok, but some more details are that the ppi is 400 and the screen to body ratio is 80%.
Gaming experience is good, but the phone gets heated up after 1 hour of gaming.
Netwok connectivity is good. Overall, it is a good phone for the price.
"""

# create a review schema

class Review(TypedDict):
    summary: str
    sentiment: str

structured_model = model.with_structured_output(Review)
response = structured_model.invoke(review)
pprint(response)

{'sentiment': 'positive',
 'summary': 'The phone offers great camera performance, long battery life, and '
            'a reasonable price, with a decent display and solid gaming '
            'experience, though it heats up after extended play. Overall, a '
            'solid value.'}


`with_structured_output` with a schema results in a system prompt behind the scenes which results in generating the response which follows the provided shema

### Using Annotations

In [80]:
class Review(TypedDict):
    summary: Annotated[str, "A summary of the review"]
    sentiment: Annotated[str, "The sentiment of the review - posutive, negative or neutral"]

structured_model = model.with_structured_output(Review)
response = structured_model.invoke(review)
pprint(response)



{'sentiment': 'positive',
 'summary': 'The phone offers a solid camera, long battery life, and a '
            'reasonable price. The display is decent with a 400\u202fppi and '
            '80% screen‑to‑body ratio. Gaming is enjoyable but the device '
            'tends to heat after an hour. Network connectivity is reliable, '
            'making it a good overall value.'}


In [81]:
# create a detailed review for iPhone 17 Air
iphone_17_air_review = """
The iPhone 17 Air represents Apple's boldest design departure in years, delivering an incredibly thin profile that feels almost impossibly light in hand. At just 5.5mm thick, this device pushes the boundaries of engineering while maintaining the premium build quality we expect from Apple. The aerospace-grade aluminum frame feels solid despite its minimal thickness, and the new Ceramic Shield front provides excellent protection without adding bulk. The device comes in four stunning colors: Midnight Black, Starlight Silver, Deep Purple, and a new Ocean Blue that shifts subtly in different lighting conditions.

Performance-wise, the iPhone 17 Air doesn't compromise despite its slim form factor. The A18 Bionic chip with its 3nm process delivers exceptional speed and efficiency, handling everything from intensive gaming to professional video editing with ease. The 8GB of unified memory ensures smooth multitasking, and the improved Neural Engine makes AI-powered features incredibly responsive. Battery life is surprisingly robust for such a thin device, easily lasting a full day of heavy usage thanks to the more efficient chip and optimized iOS 18 integration. The new MagSafe wireless charging is faster than ever, reaching 25W speeds that rival many wired chargers.

The camera system is where the iPhone 17 Air truly shines, featuring a revolutionary new periscope telephoto lens that somehow fits within the ultra-thin chassis. The main 48MP sensor captures stunning photos with incredible detail and dynamic range, while the new computational photography features produce professional-quality results in challenging lighting conditions. Night mode has been significantly improved, and the new Action mode for video recording delivers gimbal-like stabilization. The front-facing camera now supports 4K ProRes recording, making it perfect for content creators who demand the highest quality.

However, the pursuit of thinness does come with some trade-offs. The device can get noticeably warm during intensive tasks, and the reduced internal space means no room for a traditional headphone jack or even the Lightning port - it's USB-C only with wireless charging as the primary power source. The speakers, while clear, lack the depth and bass response of thicker iPhone models. Additionally, the ultra-thin design makes the device feel somewhat fragile, and Apple's recommended case adds back much of the thickness that the Air design eliminates. Despite these minor compromises, the iPhone 17 Air succeeds in creating a truly premium, futuristic smartphone experience that feels like a glimpse into the next decade of mobile technology.
"""


In [85]:
class Review(TypedDict):
    key_themes: Annotated[list[str], "The key themes of the review as a list of strings"]
    summary: Annotated[str, "A brief summary of the review - max 100 words"]
    sentiment: Annotated[Literal["pos", "neg", "neu"], "The sentiment of the review"]
    pros: Annotated[Optional[list[str]], "The pros of the review as a list of strings"]
    cons: Annotated[Optional[list[str]], "The cons of the review as a list of strings"]

structured_model = model.with_structured_output(Review)
response = structured_model.invoke(iphone_17_air_review)
pprint(response)

{'cons': ['Device can get noticeably warm during intensive tasks',
          'No headphone jack or Lightning port – USB-C only',
          'Speakers lack depth and bass response',
          'Ultra-thin design feels somewhat fragile',
          "Apple's recommended case adds back much of the thickness that the "
          'Air design eliminates'],
 'key_themes': ['Design and Build',
                'Performance and Efficiency',
                'Camera System',
                'Battery Life and Charging',
                'Trade-offs and Limitations'],
 'pros': ['Incredibly thin profile at 5.5mm',
          'Aerospace-grade aluminum frame',
          'Ceramic Shield front protection',
          'Stunning color options',
          'A18 Bionic chip with 3nm process',
          '8GB unified memory',
          'Improved Neural Engine',
          'Robust battery life',
          '25W MagSafe wireless charging',
          'Revolutionary periscope telephoto lens',
          '48MP main sensor',
 

In [ ]:
# create a detailed review for iPhone 17 Air -- without cons
iphone_17_air_review_without_cons = """
The iPhone 17 Air represents Apple's boldest design departure in years, delivering an incredibly thin profile that feels almost impossibly light in hand. At just 5.5mm thick, this device pushes the boundaries of engineering while maintaining the premium build quality we expect from Apple. The aerospace-grade aluminum frame feels solid despite its minimal thickness, and the new Ceramic Shield front provides excellent protection without adding bulk. The device comes in four stunning colors: Midnight Black, Starlight Silver, Deep Purple, and a new Ocean Blue that shifts subtly in different lighting conditions.

Performance-wise, the iPhone 17 Air doesn't compromise despite its slim form factor. The A18 Bionic chip with its 3nm process delivers exceptional speed and efficiency, handling everything from intensive gaming to professional video editing with ease. The 8GB of unified memory ensures smooth multitasking, and the improved Neural Engine makes AI-powered features incredibly responsive. Battery life is surprisingly robust for such a thin device, easily lasting a full day of heavy usage thanks to the more efficient chip and optimized iOS 18 integration. The new MagSafe wireless charging is faster than ever, reaching 25W speeds that rival many wired chargers.

The camera system is where the iPhone 17 Air truly shines, featuring a revolutionary new periscope telephoto lens that somehow fits within the ultra-thin chassis. The main 48MP sensor captures stunning photos with incredible detail and dynamic range, while the new computational photography features produce professional-quality results in challenging lighting conditions. Night mode has been significantly improved, and the new Action mode for video recording delivers gimbal-like stabilization. The front-facing camera now supports 4K ProRes recording, making it perfect for content creators who demand the highest quality.
"""

structured_model = model.with_structured_output(Review)
response = structured_model.invoke(iphone_17_air_review_without_cons)
pprint(response)



{'cons': [],
 'key_themes': ['Design & Build',
                'Performance & Efficiency',
                'Battery & Charging',
                'Camera & Photography',
                'User Experience'],
 'pros': ['Ultra-thin 5.5mm profile with premium aerospace-grade aluminum '
          'frame',
          'Exceptional performance from A18 Bionic 3nm chip and 8GB RAM',
          'Robust battery life and fast 25W MagSafe charging',
          'Revolutionary periscope telephoto lens and 48MP main sensor',
          'Advanced computational photography and improved night mode',
          'Front camera supports 4K ProRes for creators'],
 'sentiment': 'pos',
 'summary': 'The iPhone\u202f17\u202fAir redefines slimness with a 5.5\u202fmm '
            'chassis that feels surprisingly solid, thanks to aerospace‑grade '
            'aluminum and Ceramic Shield. Powered by the A18 Bionic 3\u202fnm '
            'chip and 8\u202fGB RAM, it delivers gaming‑grade speed, efficient '
            'mul

This works really well -- but there is no data validation - this can be done by Pydantic

### Using Pydantic

In [11]:
from pydantic import BaseModel

In [12]:
class Student(BaseModel):
    name: str

new_student = {'name': 'mini'}
student = Student(**new_student)
print(student)

name='mini'


In [13]:
class Student(BaseModel):
    name: str = "mini" # default value
    age: Optional[int] = None # optional field

student = Student()
print(student)

name='mini' age=None


In [14]:
# pydantic does type coercion, whenever possible
class Student(BaseModel):
    name: str = "mini"
    age: Optional[int] = None

student = Student(name="mini", age="29")
print(student)

name='mini' age=29


In [15]:
# email validation using pydantic
from pydantic import EmailStr
class User(BaseModel):
    name: str
    email: EmailStr

user = User(name="mini", email="mini@example.com")
print(user)

name='mini' email='mini@example.com'


In [98]:
user = User(name="mini", email="mini@")
print(user)

ValidationError: 1 validation error for User
email
  value is not a valid email address: There must be something after the @-sign. [type=value_error, input_value='mini@', input_type=str]

In [ ]:
# using Field in Pydantic
from pydantic import Field


class Student(BaseModel):
    name: str = "mini"
    age: Optional[int] = None
    email: EmailStr 
    cgpa: float = Field(ge=0, le=10, default=8.0, description="The CGPA of the student") # description is like the annotation in TypedDict - helps the llm understand the field

student = Student(name="mini", age=20, email="mini@example.com", cgpa=9.5)
print(student)


name='mini' age=20 email='mini@example.com' cgpa=9.5


In [18]:
student = Student(name="mini", age=20, email="mini@example.com", cgpa=9.5)
print(student)

name='mini' age=20 email='mini@example.com' cgpa=9.5


In [ ]:
dict(student)

{'name': 'mini', 'age': 20, 'email': 'mini@example.com', 'cgpa': 9.5}

In [ ]:
class Review(TypedDict):
    key_themes: Annotated[list[str], "The key themes of the review as a list of strings"]
    summary: Annotated[str, "A brief summary of the review - max 100 words"]
    sentiment: Annotated[Literal["pos", "neg", "neu"], "The sentiment of the review"]
    pros: Annotated[Optional[list[str]], "The pros of the review as a list of strings"]
    cons: Annotated[Optional[list[str]], "The cons of the review as a list of strings"]

In [22]:
# create a detailed review for iPhone 17 Air -- without cons
iphone_17_air_review_without_cons = """
The iPhone 17 Air represents Apple's boldest design departure in years, delivering an incredibly thin profile that feels almost impossibly light in hand. At just 5.5mm thick, this device pushes the boundaries of engineering while maintaining the premium build quality we expect from Apple. The aerospace-grade aluminum frame feels solid despite its minimal thickness, and the new Ceramic Shield front provides excellent protection without adding bulk. The device comes in four stunning colors: Midnight Black, Starlight Silver, Deep Purple, and a new Ocean Blue that shifts subtly in different lighting conditions.

Performance-wise, the iPhone 17 Air doesn't compromise despite its slim form factor. The A18 Bionic chip with its 3nm process delivers exceptional speed and efficiency, handling everything from intensive gaming to professional video editing with ease. The 8GB of unified memory ensures smooth multitasking, and the improved Neural Engine makes AI-powered features incredibly responsive. Battery life is surprisingly robust for such a thin device, easily lasting a full day of heavy usage thanks to the more efficient chip and optimized iOS 18 integration. The new MagSafe wireless charging is faster than ever, reaching 25W speeds that rival many wired chargers.

The camera system is where the iPhone 17 Air truly shines, featuring a revolutionary new periscope telephoto lens that somehow fits within the ultra-thin chassis. The main 48MP sensor captures stunning photos with incredible detail and dynamic range, while the new computational photography features produce professional-quality results in challenging lighting conditions. Night mode has been significantly improved, and the new Action mode for video recording delivers gimbal-like stabilization. The front-facing camera now supports 4K ProRes recording, making it perfect for content creators who demand the highest quality.
"""


class Review(BaseModel):
    key_themes: List[str] = Field(description="The key themes of the review as a list of strings")
    summary: str = Field(description="A brief summary of the review - max 100 words")
    sentiment: Literal["pos", "neg", "neu"] = Field(description="The sentiment of the review - posutive, negative or neutral")
    pros: Optional[List[str]] = Field(description="The pros of the review as a list of strings", default=None)
    cons: Optional[List[str]] = Field(description="The cons of the review as a list of strings", default=None)

structured_model = model.with_structured_output(Review)
response = structured_model.invoke(iphone_17_air_review_without_cons)
pprint(dict(response))

{'cons': [],
 'key_themes': ['design',
                'performance',
                'battery',
                'camera',
                'MagSafe',
                'color options'],
 'pros': ['ultra-thin 5.5mm profile',
          'aerospace-grade aluminum frame',
          'Ceramic Shield front',
          'four vibrant colors',
          'A18 Bionic 3nm chip',
          '8GB unified memory',
          'Neural Engine AI',
          'robust battery life',
          '25W MagSafe charging',
          'periscope telephoto lens',
          '48MP main sensor',
          'advanced computational photography',
          'improved night mode',
          'Action mode stabilization',
          '4K ProRes front camera'],
 'sentiment': 'pos',
 'summary': 'The iPhone\u202f17\u202fAir redefines slimness with a 5.5\u202fmm '
            'chassis that feels almost weightless yet feels solid thanks to '
            'aerospace‑grade aluminum and Ceramic Shield. Powered by the A18 '
            'Bionic 3

### JSON Schema

This is useful when u cannot define the structure using python - say u are restricted to a diff langauge

In [23]:
{
    "title": "Student",
    "description": "A student is a person who is studying at a school or university",
    "type": "object",
    "properties": {
        "name": {
            "type": "string",
            "description": "The name of the student"
        },
        "age": {
            "type": "integer",
            "description": "The age of the student"
        }
    },
    "required": ["name"]
}

{'title': 'Student',
 'description': 'A student is a person who is studying at a school or university',
 'type': 'object',
 'properties': {'name': {'type': 'string',
   'description': 'The name of the student'},
  'age': {'type': 'integer', 'description': 'The age of the student'}},
 'required': ['name']}

In [24]:
# review schema
# schema
json_schema = {
  "title": "Review",
  "type": "object",
  "properties": {
    "key_themes": {
      "type": "array",
      "items": {
        "type": "string"
      },
      "description": "Write down all the key themes discussed in the review in a list"
    },
    "summary": {
      "type": "string",
      "description": "A brief summary of the review"
    },
    "sentiment": {
      "type": "string",
      "enum": ["pos", "neg"],
      "description": "Return sentiment of the review either negative, positive or neutral"
    },
    "pros": {
      "type": ["array", "null"],
      "items": {
        "type": "string"
      },
      "description": "Write down all the pros inside a list"
    },
    "cons": {
      "type": ["array", "null"],
      "items": {
        "type": "string"
      },
      "description": "Write down all the cons inside a list"
    },
    "name": {
      "type": ["string", "null"],
      "description": "Write the name of the reviewer"
    }
  },
  "required": ["key_themes", "summary", "sentiment"]
}

In [25]:
structured_model = model.with_structured_output(json_schema)

response = structured_model.invoke(iphone_17_air_review_without_cons)
pprint(response)

{'cons': None,
 'key_themes': ['Design & Build',
                'Performance & Efficiency',
                'Battery Life',
                'MagSafe Charging',
                'Camera System',
                'Color Options'],
 'name': None,
 'pros': ['Ultra‑thin 5.5\u202fmm profile',
          'Lightweight feel',
          'Aerospace‑grade aluminum frame',
          'Ceramic Shield front',
          'Four vibrant color options',
          'A18 Bionic 3\u202fnm chip',
          '8\u202fGB unified memory',
          'Neural Engine AI features',
          'Full‑day battery life',
          '25\u202fW MagSafe wireless charging',
          'Periscope telephoto lens',
          '48\u202fMP main sensor',
          'Advanced computational photography',
          'Improved Night mode',
          'Action mode video stabilization',
          '4K ProRes front camera'],
 'sentiment': 'pos',
 'summary': 'The iPhone\u202f17\u202fAir delivers a bold, ultra‑thin design '
            'with premium bui

## Output Parsers

If your llm cannot generate structured ops out of the box, we can use Op Parsers -- these are classes in LangChain that help convert raw llm responses into structured ops

These can be used both with models which can and cannot provide structured ops

### StrOutputParser

In [30]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser


In [28]:
# 1st prompt -> detailed report
template1 = PromptTemplate(
    template='Write a detailed report on {topic}',
    input_variables=['topic']
)

# 2nd prompt -> summary
template2 = PromptTemplate(
    template='Write a 2 line summary on the following text. /n {text}',
    input_variables=['text']
)

parser = StrOutputParser()


In [29]:
chain = template1 | model | parser | template2 | model | parser

chain.invoke({"topic": "AI"})

'This comprehensive report traces AI’s evolution from early symbolic systems to today’s foundation models, detailing core technologies, real‑world applications, and the economic, ethical, and governance challenges they pose. It underscores AI’s transformative impact across industries while calling for responsible design, transparency, and global cooperation to harness its benefits and mitigate risks.'

### JsonOutputParser

In [34]:
JsonOutputParser().get_format_instructions()

'Return a JSON object.'

In [35]:
parser = JsonOutputParser()

template = PromptTemplate(
    template='Give me 5 facts about {topic} \n {format_instruction}',
    input_variables=['topic'],
    partial_variables={'format_instruction': parser.get_format_instructions()}
)

chain = template | model | parser

result = chain.invoke({'topic':'black hole'})

pprint(result)

{'facts': ['Black holes are regions of spacetime where gravity is so strong '
           'that nothing, not even light, can escape once it crosses the event '
           'horizon.',
           'The size of a black hole is defined by its Schwarzschild radius, '
           'which is proportional to its mass (approximately 3 kilometers per '
           'solar mass).',
           'Supermassive black holes, with masses millions to billions of '
           'times that of the Sun, reside at the centers of most galaxies, '
           'including our Milky Way.',
           'Black holes can grow by accreting matter from their surroundings '
           'or by merging with other black holes, a process that emits '
           'powerful gravitational waves detectable by observatories like LIGO '
           'and Virgo.',
           'Despite their name, black holes are not perfect vacuum; they can '
           'emit Hawking radiation—a theoretical quantum effect that causes '
           'them to lose 

Here its returning in JSON but we __cannot control the schema__

### StructuredOutputParser

- we can enforce a schema here

In [36]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

In [37]:
schema = [ResponseSchema(name="fact_1", description="The first fact about the topic"),
ResponseSchema(name="fact_2", description="The second fact about the topic"),
ResponseSchema(name="fact_3", description="The third fact about the topic"),
ResponseSchema(name="fact_4", description="The fourth fact about the topic"),
ResponseSchema(name="fact_5", description="The fifth fact about the topic")
]

parser = StructuredOutputParser.from_response_schemas(schema)

parser.get_format_instructions()

'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"fact_1": string  // The first fact about the topic\n\t"fact_2": string  // The second fact about the topic\n\t"fact_3": string  // The third fact about the topic\n\t"fact_4": string  // The fourth fact about the topic\n\t"fact_5": string  // The fifth fact about the topic\n}\n```'

In [ ]:
template = PromptTemplate(
    template='Give me 5 facts about {topic} \n {format_instruction}',
    input_variables=['topic'],
    partial_variables={'format_instruction': parser.get_format_instructions()}
)

chain = template | gemini_flash_model | parser # note: this fails with the openai oss model

chain.invoke({'topic':'black hole'})

{'fact_1': 'A black hole is a region of spacetime where gravity is so strong that nothing, not even light, can escape from it.',
 'fact_2': 'The boundary around a black hole beyond which no escape is possible is called the event horizon.',
 'fact_3': 'Most black holes form from the remnants of large stars that collapse in on themselves at the end of their life cycle, known as stellar black holes.',
 'fact_4': 'Supermassive black holes, millions to billions of times the mass of our Sun, are found at the center of most large galaxies, including our own Milky Way (Sagittarius A*).',
 'fact_5': "Despite their immense gravity, black holes do not 'suck' things in from vast distances; objects must get very close to be pulled in, and if our Sun were replaced by a black hole of the same mass, Earth would continue to orbit it normally."}

Disadv

- CANNOT do data validation 
- even if llm sends wrong format, (eg: str instead of int) we cannot validate that

### PydanticOutputParser

In [44]:
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

# define schema
class Person(BaseModel):
    name: str = Field(description="The name of the person")
    age: int = Field(description="The age of the person", gt=18, lt=150),
    city: str = Field(description="The city of the person")

# create parser
parser = PydanticOutputParser(pydantic_object=Person)


In [45]:
parser.get_format_instructions()

/Users/shaunak.sen/Documents/shaunak-dev/LLM-Engineering/.venv/lib/python3.12/site-packages/pydantic/json_schema.py:2324: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, description='The age of the person', metadata=[Gt(gt=18), Lt(lt=150)]),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)


'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"description": "The name of the person", "title": "Name", "type": "string"}, "age": {"title": "Age", "type": "integer"}, "city": {"description": "The city of the person", "title": "City", "type": "string"}}, "required": ["name", "city"]}\n```'

In [ ]:
template = PromptTemplate(
    template='Give me the name, age and city of the person with details: {details} \n {format_instruction}',
    input_variables=['details'],
    partial_variables={'format_instruction': parser.get_format_instructions()}
)

print (template.invoke({'details':'John is 25 years old and lives in New York'}))

text='Give me the name, age and city of the person with details: John is 25 years old and lives in New York \n The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"description": "The name of the person", "title": "Name", "type": "string"}, "age": {"title": "Age", "type": "integer"}, "city": {"description": "The city of the person", "title": "City", "type": "string"}}, "required": ["name", "city"]}\n```'


/Users/shaunak.sen/Documents/shaunak-dev/LLM-Engineering/.venv/lib/python3.12/site-packages/pydantic/json_schema.py:2324: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, description='The age of the person', metadata=[Gt(gt=18), Lt(lt=150)]),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)


In [50]:
chain = template | model | parser

chain.invoke({'details':'John is 25 years old and lives in New York'})

Person(name='John', age=25, city='New York')

## Chains in LangChain

### Sequential Chain

In [54]:
prompt = PromptTemplate(
    template='Generate 5 interesting facts about {topic}',
    input_variables=['topic']
)

chain = prompt | model | StrOutputParser()

print(chain.invoke({'topic':'black hole'}))

**Five Fascinating Facts About Black Holes**

1. **They’re Not “Vacuum Cleaners”**  
   Despite the popular image of a black hole sucking in everything nearby, space around a black hole is essentially empty. Objects only fall in if they cross the event horizon or are on a trajectory that brings them close enough to be captured by the black hole’s gravity.

2. **Time Slows Down Near the Event Horizon**  
   According to Einstein’s theory of relativity, time runs slower the closer you are to a massive object. Near a black hole’s event horizon, time can dilate to the point where, for an outside observer, an infalling object appears to freeze and fade as it approaches the horizon.

3. **They Emit Hawking Radiation**  
   In 1974, Stephen Hawking predicted that black holes can emit tiny amounts of thermal radiation due to quantum effects near the event horizon. This “Hawking radiation” means black holes can slowly lose mass and eventually evaporate over astronomically long timescales.

4. *

In [56]:
chain.get_graph().print_ascii()

     +-------------+       
     | PromptInput |       
     +-------------+       
            *              
            *              
            *              
    +----------------+     
    | PromptTemplate |     
    +----------------+     
            *              
            *              
            *              
      +----------+         
      | ChatGroq |         
      +----------+         
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *              
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  


In [57]:
prompt_detailed_report = PromptTemplate(
    template='Write a detailed report on {topic}',
    input_variables=['topic']
)

prompt_key_facts = PromptTemplate(
    template='Give me a 3 point summary of the following text: {text}',
    input_variables=['text']
)

chain = prompt_detailed_report | model | StrOutputParser() | prompt_key_facts | model | StrOutputParser()


print(chain.invoke({'topic':'cricket'})) 

**Three‑point summary**

1. **Cricket’s structure and reach** – A bat‑and‑ball sport governed by the ICC, played worldwide by over 2.5 billion fans. It exists in five main formats (Test, ODI, T20I, The Hundred, domestic first‑class), each with distinct rules, durations, and audiences.

2. **Economic and cultural significance** – Cricket generates a multi‑billion‑dollar global market (e.g., IPL >US$1 billion in 2023). It is a national pastime in South Asia, the Caribbean, Australia, England, and New Zealand, shaping identity, media, and even diplomatic relations.

3. **Future trajectory** – Technological innovations (DRS, AI analytics, smart gear), new formats (100‑over matches, hybrid games), and sustainability initiatives are reshaping the sport, while expansion into emerging markets (US, China, Africa) and governance reforms aim to balance commercial growth with tradition.


In [58]:
chain.get_graph().print_ascii()

     +-------------+       
     | PromptInput |       
     +-------------+       
            *              
            *              
            *              
    +----------------+     
    | PromptTemplate |     
    +----------------+     
            *              
            *              
            *              
      +----------+         
      | ChatGroq |         
      +----------+         
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *              
    +----------------+     
    | PromptTemplate |     
    +----------------+     
            *              
            *              
            *              
      +----------+         
      | ChatGroq |         
      +----------+         
            *              
            *              
            *       

### Parallel Chain

In [59]:
prompt_1 = PromptTemplate(
    template="Generate short and simple notes from the following text: {text}",
    input_variables=['text']
)

prompt_2 = PromptTemplate(
    template="Generate 5 short simple QnAs like a quiz from the following text: {text}",
    input_variables=['text']
)


prompt_3 = PromptTemplate(
    template="Merge the provided notes and quiz into a single document \n {notes} \n {quiz}",
    input_variables=['notes', 'quiz']
)

parser = StrOutputParser()

In [60]:
from langchain.schema.runnable import RunnableParallel

In [63]:
text = model.invoke("Generate a detailed report on Attention is all you need paper").content

In [64]:
print (text)

# Attention Is All You Need  
**Vaswani, A., Shazeer, N., Parmar, N., Uszkoreit, J., Jones, L., Gomez, A. N., Kaiser, Ł., & Polosukhin, I. (2017).** *Attention Is All You Need*. In *Advances in Neural Information Processing Systems* (NeurIPS 2017).  

---

## 1. Executive Summary  

The 2017 NeurIPS paper “Attention Is All You Need” introduced the **Transformer** architecture, a novel neural network model that dispenses with recurrence and convolution entirely, relying solely on a **self‑attention** mechanism. The Transformer achieved state‑of‑the‑art results on several machine‑translation benchmarks (WMT 2014 English‑German and English‑French) while dramatically reducing training time and enabling far larger parallelism. Its design has since become the backbone of virtually all modern large‑scale language models (BERT, GPT, T5, etc.).

Key innovations:

| Innovation | What it solves | Impact |
|------------|----------------|--------|
| **Scaled Dot‑Product Attention** | Efficient, dif

In [65]:
# define the parallel chain

parallel_chain = RunnableParallel({
    "notes": prompt_1 | model | StrOutputParser(),
    "quiz": prompt_2 | model | StrOutputParser(),
})

# define the sequential chain for merging

merge_chain = prompt_3 | model | parser

# combine the parallel and sequential chains

chain = parallel_chain | merge_chain

print(chain.invoke({"text":text}))


# “Attention Is All You Need” – Short & Simple Notes + Quiz  
*(Vaswani et al., 2017 – NeurIPS)*  

---

## 1. What It Is  
The **Transformer** – a neural‑network architecture that relies **only on self‑attention** (no RNNs or CNNs). It became the foundation for BERT, GPT, T5, and many other state‑of‑the‑art language models.

---

## 2. Why It Matters  

| Benefit | Why It Matters |
|---------|----------------|
| **Faster training** | Fully parallelizable – no sequential bottleneck. |
| **Higher accuracy** | Outperformed RNN/CNN baselines on WMT 2014 (BLEU 28.4 EN‑DE, 41.0 EN‑FR). |
| **Scalable** | Works on GPUs/TPUs; training time grows linearly with GPU count. |
| **Modular** | Easy to stack, adapt, or extend (e.g., BERT, GPT). |

---

## 3. Core Ideas  

| Idea | Purpose | Effect |
|------|---------|--------|
| **Scaled Dot‑Product Attention** | Efficiently weighs token relationships | Core building block |
| **Multi‑Head Attention** | Learns multiple “views” of the data | Richer c

In [66]:
chain.get_graph().print_ascii()

          +---------------------------+            
          | Parallel<notes,quiz>Input |            
          +---------------------------+            
                ***             ***                
              **                   **              
            **                       **            
+----------------+              +----------------+ 
| PromptTemplate |              | PromptTemplate | 
+----------------+              +----------------+ 
          *                             *          
          *                             *          
          *                             *          
    +----------+                  +----------+     
    | ChatGroq |                  | ChatGroq |     
    +----------+                  +----------+     
          *                             *          
          *                             *          
          *                             *          
+-----------------+            +-----------------+ 
| StrOutputP

### Conditional Chain

---- see notes

In [73]:
from langchain.schema.runnable import RunnableBranch, RunnableLambda

In [69]:
class Feedback(BaseModel):
    sentiment: Literal["positive", "negative"] = Field(description="The sentiment of the text")

In [70]:
parser = PydanticOutputParser(pydantic_object=Feedback)

In [71]:
prompt_1 = PromptTemplate(
    template="Classify the sentiment of the following text in either positive or negative: {text} \n {format_instruction}",
    input_variables=['text'],
    partial_variables={'format_instruction': parser.get_format_instructions()}
)

chain = prompt_1 | model | parser

print(chain.invoke({"text":"I am happy"}))

sentiment='positive'


#### Conditional Chain structure

```
branch_chain = RunnableBranch(
    (condition_1, chain_2),
    (condition_2, chain_2),
    ...
    default chain if no condition is met
)
```

Its like an __if elif ... else__ statement

In [75]:
prompt_2 =  PromptTemplate(
    template="Write an appropriate follow up response to the positive feedback: {feedback}",
    input_variables=['feedback']
)
prompt_3 =  PromptTemplate(
    template="Write an appropriate follow up response to the negative feedback: {feedback}",
    input_variables=['feedback']
)


branch_chain = RunnableBranch(
   (lambda x: x.sentiment == "positive", prompt_2 | model | StrOutputParser()),
   (lambda x: x.sentiment == "negative", prompt_3 | model | StrOutputParser()),
   RunnableLambda(lambda x: "Could not understand the sentiment") # why use RunnableLambda? It is to convert the lambda function to a Runnable chain
)

chain = prompt_1 | model | parser | branch_chain


In [77]:
print(chain.invoke({"text":"The new phone is great"}))

Thank you so much for your kind words! I’m thrilled to hear you had a great experience. If there’s anything else I can help with or if you have any suggestions for improvement, please let me know. Your feedback truly makes a difference!


In [78]:
print(chain.invoke({"text":"I want to return this product ASAP"}))

Thank you for sharing your experience. I’m sorry to hear that things didn’t meet your expectations. Your feedback is important to us, and I’d like to understand what went wrong so we can make things right. Could you please provide a bit more detail about the issue you encountered? Once I have that information, I’ll do my best to resolve it promptly. Thank you for giving us the chance to improve.


In [79]:
chain.get_graph().print_ascii()

    +-------------+      
    | PromptInput |      
    +-------------+      
            *            
            *            
            *            
   +----------------+    
   | PromptTemplate |    
   +----------------+    
            *            
            *            
            *            
      +----------+       
      | ChatGroq |       
      +----------+       
            *            
            *            
            *            
+----------------------+ 
| PydanticOutputParser | 
+----------------------+ 
            *            
            *            
            *            
       +--------+        
       | Branch |        
       +--------+        
            *            
            *            
            *            
    +--------------+     
    | BranchOutput |     
    +--------------+     


### Runnables in LangChain


--- see notes

In [26]:
# component 1

class DummyLLM():

    def __init__(self):
        logger.info("Initializing DummyLLM...")

    def predict(self, prompt):

        logger.info(f"Predicting with prompt: {prompt}")

        dummy_responses = [
            "This is a dummy response",
            "This is another dummy response",
            "This is yet another dummy response with a different format"
        ]

        return {"response": random.choice(dummy_responses)}
    
    
dummy_llm = DummyLLM()

print(dummy_llm.predict("Hello"))

2025-09-20 10:08:48.445 | INFO     | __main__:__init__:6 - Initializing DummyLLM...
2025-09-20 10:08:48.445 | INFO     | __main__:predict:10 - Predicting with prompt: Hello


{'response': 'This is a dummy response'}


In [27]:
# component 2

class DummyPromptTemplate():

    def __init__(self, template, input_variables):
        logger.info("Initializing DummyPromptTemplate...")
        self.template = template
        self.input_variables = input_variables

    def format(self, input_dict):
        return self.template.format(**input_dict)

In [28]:
template = DummyPromptTemplate(template="Hello {name}, I am {age} years old", input_variables=["name", "age"])

print(template.format({"name":"John", "age":25}))

2025-09-20 10:08:48.879 | INFO     | __main__:__init__:6 - Initializing DummyPromptTemplate...


Hello John, I am 25 years old


In [29]:
# now we can use these dummy components to create a simple application

template = DummyPromptTemplate(template="Tell me about topic: {topic}", input_variables=["topic"])
prompt = template.format({"topic":"AI"})
response = dummy_llm.predict(prompt)
print(response)
        

2025-09-20 10:08:51.198 | INFO     | __main__:__init__:6 - Initializing DummyPromptTemplate...
2025-09-20 10:08:51.199 | INFO     | __main__:predict:10 - Predicting with prompt: Tell me about topic: AI


{'response': 'This is yet another dummy response with a different format'}


In [30]:
# now lets see how chains used to work in LangChain

class DummyChain():

    def __init__(self, llm, prompt):
        logger.info("Initializing DummyChain...")
        self.llm = llm
        self.prompt = prompt

    def run(self, input_dict):
        prompt = self.prompt.format(input_dict)
        response = self.llm.predict(prompt)
        return response



template = DummyPromptTemplate(template="Tell me about topic: {topic}", input_variables=["topic"])
dummy_llm = DummyLLM()

chain = DummyChain(dummy_llm, template)


2025-09-20 10:09:30.897 | INFO     | __main__:__init__:6 - Initializing DummyPromptTemplate...
2025-09-20 10:09:30.897 | INFO     | __main__:__init__:6 - Initializing DummyLLM...
2025-09-20 10:09:30.898 | INFO     | __main__:__init__:6 - Initializing DummyChain...


In [31]:
chain.run({"topic":"AI"})

2025-09-20 10:09:32.745 | INFO     | __main__:predict:10 - Predicting with prompt: Tell me about topic: AI


{'response': 'This is yet another dummy response with a different format'}

Note here the `.run` method in DummyChain can work with any prompt and LLM -- but what if we need mult llm calls? this cannot be configured easily

#### Using standardized components

In [ ]:
# component 1

class DummyLLM():

    def __init__(self):
        logger.info("Initializing DummyLLM...")

    def predict(self, prompt):

        logger.info(f"Predicting with prompt: {prompt}")

        dummy_responses = [
            "This is a dummy response",
            "This is another dummy response",
            "This is yet another dummy response with a different format"
        ]

        return {"response": random.choice(dummy_responses)}


class DummyPromptTemplate():

    def __init__(self, template, input_variables):
        logger.info("Initializing DummyPromptTemplate...")
        self.template = template
        self.input_variables = input_variables

    def format(self, input_dict):
        return self.template.format(**input_dict)
    

Now in order to make these standardized:

- we need to convert these to runnables
- each runnable should have a common method: `invoke()`

We can do this using __Abstraction__

In [32]:
from abc import ABC, abstractmethod

class Runnable(ABC):

    @abstractmethod
    def invoke(self, input):
        pass

In [33]:
class DummyLLM(Runnable):

    def __init__(self):
        logger.info("Initializing DummyLLM...")

    def predict(self, prompt):

        logger.info(f"Predicting with prompt: {prompt}")

        dummy_responses = [
            "This is a dummy response",
            "This is another dummy response",
            "This is yet another dummy response with a different format"
        ]

        return {"response": random.choice(dummy_responses)}

dummy_llm = DummyLLM()

TypeError: Can't instantiate abstract class DummyLLM without an implementation for abstract method 'invoke'

__So now we are forced to implement the invoke methods for all classes that inherit from `Runnable`__

In [ ]:
class DummyLLM(Runnable):

    def __init__(self):
        logger.info("Initializing DummyLLM...")

    def predict(self, prompt):
        # NOTE: this is the old method
        logger.warning("This method is deprecated. Please use invoke() instead.")
        logger.info(f"Predicting with prompt: {prompt}")

        dummy_responses = [
            "This is a dummy response",
            "This is another dummy response",
            "This is yet another dummy response with a different format"
        ]

        return {"response": random.choice(dummy_responses)}

    def invoke(self, prompt):

        logger.info(f"Predicting with prompt: {prompt}")

        dummy_responses = [
            "This is a dummy response",
            "This is another dummy response",
            "This is yet another dummy response with a different format"
        ]

        return {"response": random.choice(dummy_responses)}

dummy_llm = DummyLLM()

2025-09-20 10:14:39.016 | INFO     | __main__:__init__:4 - Initializing DummyLLM...


In [35]:
dummy_llm.invoke("Hello")

2025-09-20 10:14:59.630 | INFO     | __main__:invoke:21 - Predicting with prompt: Hello


{'response': 'This is yet another dummy response with a different format'}

In [36]:
dummy_llm.predict("Hello")

2025-09-20 10:15:06.438 | WARNING  | __main__:predict:8 - This method is deprecated. Please use invoke() instead.
2025-09-20 10:15:06.439 | INFO     | __main__:predict:9 - Predicting with prompt: Hello


{'response': 'This is yet another dummy response with a different format'}

In [45]:
class DummyPromptTemplate(Runnable):

    def __init__(self, template, input_variables):
        logger.info("Initializing DummyPromptTemplate...")
        self.template = template
        self.input_variables = input_variables

    def format(self, input_dict):
        # NOTE: this is the old method
        logger.warning("This method is deprecated. Please use invoke() instead.")
        return self.template.format(**input_dict)
    
    def invoke(self, input_dict):
        return self.template.format(**input_dict)

Now that we have standardized the components, we can build a chain

In [50]:
class RunnableConnector(Runnable):
    """
    This is a connector that can be used to connect multiple runnables in a chain
    """

    def __init__(self, runnable_list: list[Runnable]) -> None:
        self.runnable_list = runnable_list

    def invoke(self, input_data):

        # for each runnable in the list, invoke it with the input data
        for runnable in self.runnable_list:
            logger.debug(f"Invoking {runnable.__class__.__name__} with input: {input_data}")
            input_data = runnable.invoke(input_data)
        return input_data

In [51]:
template = DummyPromptTemplate(template="Tell me about topic: {topic}", input_variables=["topic"])
dummy_llm = DummyLLM()

2025-09-20 10:22:28.505 | INFO     | __main__:__init__:4 - Initializing DummyPromptTemplate...
2025-09-20 10:22:28.505 | INFO     | __main__:__init__:4 - Initializing DummyLLM...


In [52]:
chain = RunnableConnector([template, dummy_llm])

chain.invoke({"topic":"AI"})

2025-09-20 10:22:28.701 | DEBUG    | __main__:invoke:13 - Invoking DummyPromptTemplate with input: {'topic': 'AI'}
2025-09-20 10:22:28.702 | DEBUG    | __main__:invoke:13 - Invoking DummyLLM with input: Tell me about topic: AI
2025-09-20 10:22:28.702 | INFO     | __main__:invoke:21 - Predicting with prompt: Tell me about topic: AI


{'response': 'This is another dummy response'}

Now lets build a new component and chain these:

In [53]:
class DummyStrOutputParser(Runnable):

    def __init__(self):
        logger.info("Initializing DummyStrOutputParser...")

    def invoke(self, input_data):
        return input_data["response"]

In [54]:
parser = DummyStrOutputParser()
chain = RunnableConnector([template, dummy_llm, parser])

chain.invoke({"topic":"AI"})

2025-09-20 10:27:27.119 | INFO     | __main__:__init__:4 - Initializing DummyStrOutputParser...
2025-09-20 10:27:27.120 | DEBUG    | __main__:invoke:13 - Invoking DummyPromptTemplate with input: {'topic': 'AI'}
2025-09-20 10:27:27.120 | DEBUG    | __main__:invoke:13 - Invoking DummyLLM with input: Tell me about topic: AI
2025-09-20 10:27:27.120 | INFO     | __main__:invoke:21 - Predicting with prompt: Tell me about topic: AI
2025-09-20 10:27:27.120 | DEBUG    | __main__:invoke:13 - Invoking DummyStrOutputParser with input: {'response': 'This is yet another dummy response with a different format'}


'This is yet another dummy response with a different format'

> Note how its so easy now to connect different components

#### Connecting multiple chains together -- result will also be a runnable


Chain1: generate joke about topic

Chain2: takes joke and creates an explanation


Connect these 2 chains

In [55]:
template1 = DummyPromptTemplate(template="Generate a joke about topic: {topic}", input_variables=["topic"])
template2 = DummyPromptTemplate(template="Create an explanation for the joke: {response}", input_variables=["response"])

dummy_llm = DummyLLM()

dummy_parser = DummyStrOutputParser()

2025-09-20 10:30:09.143 | INFO     | __main__:__init__:4 - Initializing DummyPromptTemplate...
2025-09-20 10:30:09.144 | INFO     | __main__:__init__:4 - Initializing DummyPromptTemplate...
2025-09-20 10:30:09.144 | INFO     | __main__:__init__:4 - Initializing DummyLLM...
2025-09-20 10:30:09.144 | INFO     | __main__:__init__:4 - Initializing DummyStrOutputParser...


In [56]:
chain1 = RunnableConnector([template1, dummy_llm])

chain1.invoke({"topic":"AI"})

2025-09-20 10:30:20.238 | DEBUG    | __main__:invoke:13 - Invoking DummyPromptTemplate with input: {'topic': 'AI'}
2025-09-20 10:30:20.238 | DEBUG    | __main__:invoke:13 - Invoking DummyLLM with input: Generate a joke about topic: AI
2025-09-20 10:30:20.238 | INFO     | __main__:invoke:21 - Predicting with prompt: Generate a joke about topic: AI


{'response': 'This is yet another dummy response with a different format'}

In [58]:
chain2 = RunnableConnector([template2, dummy_llm, dummy_parser])

chain2.invoke({"response":"This is a joke about AI"})

2025-09-20 10:30:52.746 | DEBUG    | __main__:invoke:13 - Invoking DummyPromptTemplate with input: {'response': 'This is a joke about AI'}
2025-09-20 10:30:52.746 | DEBUG    | __main__:invoke:13 - Invoking DummyLLM with input: Create an explanation for the joke: This is a joke about AI
2025-09-20 10:30:52.747 | INFO     | __main__:invoke:21 - Predicting with prompt: Create an explanation for the joke: This is a joke about AI
2025-09-20 10:30:52.747 | DEBUG    | __main__:invoke:13 - Invoking DummyStrOutputParser with input: {'response': 'This is another dummy response'}


'This is another dummy response'

In [59]:
consolidated_chain = RunnableConnector([chain1, chain2])

consolidated_chain.invoke({"topic":"AI"})

2025-09-20 10:31:14.258 | DEBUG    | __main__:invoke:13 - Invoking RunnableConnector with input: {'topic': 'AI'}
2025-09-20 10:31:14.259 | DEBUG    | __main__:invoke:13 - Invoking DummyPromptTemplate with input: {'topic': 'AI'}
2025-09-20 10:31:14.259 | DEBUG    | __main__:invoke:13 - Invoking DummyLLM with input: Generate a joke about topic: AI
2025-09-20 10:31:14.259 | INFO     | __main__:invoke:21 - Predicting with prompt: Generate a joke about topic: AI
2025-09-20 10:31:14.260 | DEBUG    | __main__:invoke:13 - Invoking RunnableConnector with input: {'response': 'This is a dummy response'}
2025-09-20 10:31:14.260 | DEBUG    | __main__:invoke:13 - Invoking DummyPromptTemplate with input: {'response': 'This is a dummy response'}
2025-09-20 10:31:14.260 | DEBUG    | __main__:invoke:13 - Invoking DummyLLM with input: Create an explanation for the joke: This is a dummy response
2025-09-20 10:31:14.260 | INFO     | __main__:invoke:21 - Predicting with prompt: Create an explanation for the

'This is another dummy response'

### Runnables - more details

#### RunnableSequence

In [13]:
from langchain.schema.runnable import RunnableSequence, RunnableParallel, RunnablePassthrough, RunnableLambda, RunnableBranch
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser


In [68]:
prompt = PromptTemplate(template="Tell me a joke about topic: {topic}", input_variables=["topic"])
parser = StrOutputParser()

chain = RunnableSequence(prompt, model, parser)

chain.invoke({"topic":"AI"})

'Why did the AI go to therapy?  \n\nBecause it had too many *neural* issues and kept *overfitting* on its past!'

#### RunnableParallel

- allows multiple runnables to execute in parallel
- each runnable receives the same input and processes it independently, producing a dict of ops

In [9]:
prompt1 = PromptTemplate(template="Generate a short tweet about topic: {topic}", input_variables=["topic"])
prompt2 = PromptTemplate(template="Generate a short linkedin post about topic: {topic}", input_variables=["topic"])

parser = StrOutputParser()

parallel_chain = RunnableParallel({
    "tweet": RunnableSequence(prompt1, model, parser),
    "linkedin": RunnableSequence(prompt2, model, parser)
})

result = parallel_chain.invoke({"topic":"Langchain"})

pprint(result)

{'linkedin': '🚀 **Unlocking the Power of LangChain** 🚀  \n'
             '\n'
             'LangChain is redefining how we build AI‑powered applications by '
             'turning large language models into modular, composable '
             'components. Whether you’re prototyping a chatbot, automating '
             'data pipelines, or creating custom LLM workflows, LangChain '
             'gives you the flexibility to mix and match tools, integrate '
             'APIs, and scale with confidence.  \n'
             '\n'
             'Curious how it can accelerate your next project? Let’s connect '
             'and explore the possibilities!  \n'
             '\n'
             '#LangChain #AI #MachineLearning #LLM #TechInnovation '
             '#DataScience #OpenSource 🚀',
 'tweet': 'Just built a chatbot with LangChain! 🚀 The modular approach makes '
          'LLM integration a breeze. #LangChain #AI #LLM #OpenAI'}


In [11]:
parallel_chain.get_graph().print_ascii()

        +-------------------------------+          
        | Parallel<tweet,linkedin>Input |          
        +-------------------------------+          
                ***             ***                
              **                   **              
            **                       **            
+----------------+              +----------------+ 
| PromptTemplate |              | PromptTemplate | 
+----------------+              +----------------+ 
          *                             *          
          *                             *          
          *                             *          
    +----------+                  +----------+     
    | ChatGroq |                  | ChatGroq |     
    +----------+                  +----------+     
          *                             *          
          *                             *          
          *                             *          
+-----------------+            +-----------------+ 
| StrOutputP

#### RunnablePassthrough

- returns the input as output without modifying it

In [ ]:
passthrough = RunnablePassthrough()

passthrough.invoke({"topic":"Langchain"}) # returns the input as output without modifying it

{'topic': 'Langchain'}

In [82]:
prompt1 = PromptTemplate(template="Write a joke about topic: {topic}", input_variables=["topic"])
parser = StrOutputParser()

joke_generator_chain = RunnableSequence(prompt1, model, parser)
prompt2 = PromptTemplate(template="Write a short explanation for the joke: {joke}", input_variables=["joke"])
joke_explanation_chain = RunnableSequence(prompt2, model, parser)

parallel_chain = RunnableParallel({
    "joke": RunnablePassthrough(),
    "explanation": joke_explanation_chain
})

final_chain = RunnableSequence(joke_generator_chain, parallel_chain)

result = final_chain.invoke({"topic":"Harry Potter"})

pprint(result)

{'explanation': '**Explanation**\n'
                '\n'
                'The joke plays on two meanings of the word *spell* and on '
                'Harry Potter’s iconic broomstick.\n'
                '\n'
                '1. **Broom‑stick loaf** – In the wizarding world a '
                '*broomstick* is the magical flying stick Harry uses. The '
                'bakery’s loaf is shaped like a broomstick, so the joke '
                'imagines a literal “broom‑stick” bread.\n'
                '\n'
                '2. **Spell out a good breakfast** – In everyday English '
                '*spell* means to write out letters. In the wizarding world it '
                'means to cast a magic spell. Harry wants to “spell” (i.e., '
                'write or cast) a good breakfast, so he goes to the bakery to '
                'get the broom‑stick loaf.\n'
                '\n'
                'Thus the humor comes from the double‑meaning of *spell* and '
                'the visual pu

#### RunnableLambda

In [96]:
def word_count(text):
    return len(text.split(" "))

runnable_word_count = RunnableLambda(word_count)

runnable_word_count.invoke("Hello world")

2

In [ ]:
prompt1 = PromptTemplate(template="Write a joke about topic: {topic}", input_variables=["topic"])
parser = StrOutputParser()

joke_generator_chain = RunnableSequence(prompt1, model, parser)

parallel_chain = RunnableParallel({
    "joke": RunnablePassthrough(),
    "num_words": RunnableLambda(lambda x: len(x.split(" ")))    
})

final_chain = RunnableSequence(joke_generator_chain, parallel_chain)

result = final_chain.invoke({"topic":"AI"})

pprint(result)

{'joke': 'Why did the AI bring a ladder to the data center?\n'
         '\n'
         'Because it heard the cloud was *high* and wanted to *scale* its own '
         'expectations!',
 'num_words': 24}


In [12]:
prompt1 = PromptTemplate(template="Write a joke about topic: {topic}", input_variables=["topic"])
parser = StrOutputParser()
joke_generator_chain = RunnableSequence(prompt1, model, parser)

prompt2 = PromptTemplate(template="Write a short explanation for the joke: {joke}", input_variables=["joke"])
joke_explanation_chain = RunnableSequence(prompt2, model, parser)

parallel_chain_1 = RunnableParallel({
    "joke": RunnablePassthrough(),
    "num_words_in_joke": RunnableLambda(lambda x: len(x.split(" "))),
    "explanation": joke_explanation_chain
})

parallel_chain_2 = RunnableParallel({
    "explanation": RunnableLambda(lambda x: x['explanation']),
    "joke": RunnableLambda(lambda x: x['joke']),
    "num_words_in_explanation": RunnableLambda(lambda x: len(x['explanation'].split(" "))),
    "num_words_in_joke": RunnableLambda(lambda x: x['num_words_in_joke']),
})

final_chain = RunnableSequence(joke_generator_chain, parallel_chain_1, parallel_chain_2)

result = final_chain.invoke({"topic":"AI"})

pprint(result)

{'explanation': '**Explanation**\n'
                '\n'
                'The joke plays on two common cloud‑computing terms that sound '
                'like everyday words:\n'
                '\n'
                '1. **“High”** – In the cloud world, “high” often refers to '
                '*high‑availability* or *high‑performance* services.  \n'
                '2. **“Scale”** – “Scaling” means adding more compute or '
                'storage resources to handle more load.\n'
                '\n'
                'The punchline turns those technical phrases into literal '
                'actions: the AI thinks the cloud is a *high* place it can '
                'climb to, so it brings a ladder. And because it wants to '
                '“scale” its own expectations, it’s literally trying to reach '
                'higher. The humor comes from treating abstract IT concepts as '
                'if they were physical objects that need a ladder.',
 'joke': 'Why did the AI bring a l

#### RunnableBranch

- like if else statement

In [31]:
parser = StrOutputParser()

email_classifier_prompt = PromptTemplate(template="Classify the email into one of the following categories: complaint, refund request, general suggestion. Only return the category name, no other text.\n\nEmail: {email}", input_variables=["email"])

email_classifier_chain = RunnableSequence(email_classifier_prompt, model, parser, RunnableLambda(lambda x: x.lower().strip()))

email_classifier_chain.invoke({"email":"good product"})

'general suggestion'

In [32]:
branch_chain = RunnableBranch(
    (lambda x: "complaint" in x, RunnableLambda(lambda x: "Oh I am sorry, we will try to fix it")),
    (lambda x: "refund" in x, RunnableLambda(lambda x: "I am sorry, we cannot refund the product")),
    (lambda x: "general" in x, RunnableLambda(lambda x: "Thank you for your email")),
    RunnablePassthrough()
)

final_chain = RunnableSequence(email_classifier_chain, branch_chain)

final_chain.invoke({"email":"I am very unhappy with the product and I want a refund"})

'I am sorry, we cannot refund the product'

In [33]:
final_chain.invoke({"email":"great product"})

'Thank you for your email'

### Tools in Langchain

#### Built in tools

In [35]:
from langchain.tools import DuckDuckGoSearchRun

In [ ]:
search_tool = DuckDuckGoSearchRun()
results = search_tool.invoke("Who will play the asia cup final")

In [40]:
pprint(results)

('1 hour ago · Asia Cup 2025 final date, teams qualified, live timings and '
 'streaming Bangladesh and Pakistan are playing out a thrilling final Super 4 '
 'tie which will make or break their campaign this year. 1 day ago · Which '
 'teams can still qualify for the Asia Cup 2025 final , and how? Al Jazeera '
 'breaks down the qualification scenarios for all four teams before the Asia '
 'Cup final in Dubai on Sunday. 50 minutes ago — Pakistan defeated Bangladesh '
 'by 11 runs in their Super Fours match in Dubai on Thursday to advance to the '
 'final of the Asia Cup 2025, where it will meet ... 18 hours ago — The winner '
 'of the match between Bangladesh and Pakistan on Thursday will qualify for '
 "the final; India's win meant Pakistan's fortunes remained in their hands ... "
 '52 minutes ago · India and Pakistan will play each other in the final of the '
 '2025 Asia Cup in a historic fixture, which will see the two teams go '
 'head-to-head in the summit clash of the continental 

In [51]:
print (search_tool.name)
print (search_tool.description)
print (search_tool.args)

duckduckgo_search
A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.
{'query': {'description': 'search query to look up', 'title': 'Query', 'type': 'string'}}


In [43]:
from langchain_community.tools import ShellTool

shell_tool = ShellTool()

shell_tool.invoke("whoami")

Executing command:
 whoami


/Users/shaunak.sen/Documents/shaunak-dev/LLM-Engineering/.venv/lib/python3.12/site-packages/langchain_community/tools/shell/tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


'shaunak.sen\n'

#### Custom Tools - using @tool

In [8]:
from langchain_core.tools import tool

@tool
def multiply(a: int, b: int) -> int:
    """
    Multiply two integers
    """
    return a * b

result = multiply.invoke({"a": 2, "b": 3})

result

6

In [49]:
print (multiply.name)
print (multiply.description)
print (multiply.args)

multiply
Multiply two integers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


Below is what the LLM sees as the tool information

In [55]:
pprint (multiply.args_schema.model_json_schema())

{'description': 'Multiply two integers',
 'properties': {'a': {'title': 'A', 'type': 'integer'},
                'b': {'title': 'B', 'type': 'integer'}},
 'required': ['a', 'b'],
 'title': 'multiply',
 'type': 'object'}


#### Using StructuredTool and Pydantic


- the argument types are strictly enforced here

In [ ]:
from langchain.tools import StructuredTool
from pydantic import BaseModel, Field

# arg schema using pydantic
class MultiplyInput(BaseModel):
    a: int = Field(description="The first number to multiply")
    b: int = Field(description="The second number to multiply")

In [57]:
def multiply_func(a: int, b: int) -> int:
    return a * b

multiply_tool = StructuredTool.from_function(
    func=multiply_func,
    name="multiply",
    description="Multiply two numbers",
    args_schema=MultiplyInput
)

result = multiply_tool.invoke({"a": 2, "b": 3})

pprint(result)

6


In [58]:
print (multiply_tool.name)
print (multiply_tool.description)
print (multiply_tool.args)

multiply
Multiply two numbers
{'a': {'description': 'The first number to multiply', 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to multiply', 'title': 'B', 'type': 'integer'}}


In [59]:
pprint (multiply_tool.args_schema.model_json_schema())

{'properties': {'a': {'description': 'The first number to multiply',
                      'title': 'A',
                      'type': 'integer'},
                'b': {'description': 'The second number to multiply',
                      'title': 'B',
                      'type': 'integer'}},
 'required': ['a', 'b'],
 'title': 'MultiplyInput',
 'type': 'object'}


#### Using BaseTool

- most flexible method

In [60]:
from langchain.tools import BaseTool
from typing import Type

In [61]:
# arg schema using pydantic
class MultiplyInput(BaseModel):
    a: int = Field(description="The first number to multiply")
    b: int = Field(description="The second number to multiply")

In [64]:
class MultiplyTool(BaseTool):
    name: str = "multiply"
    description: str = "Multiply two numbers"
    args_schema: Type[BaseModel] = MultiplyInput

    def _run(self, a: int, b: int) -> int:
        return a * b

    def _arun(self, a: int, b: int) -> int: # you can implement an async version of the run method
        pass

multiply_tool = MultiplyTool()

result = multiply_tool.invoke({"a": 2, "b": 3})

pprint(result)
print (multiply_tool.name)
print (multiply_tool.description)
print (multiply_tool.args)

6
multiply
Multiply two numbers
{'a': {'description': 'The first number to multiply', 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to multiply', 'title': 'B', 'type': 'integer'}}


In [63]:
pprint (multiply_tool.args_schema.model_json_schema())

{'properties': {'a': {'description': 'The first number to multiply',
                      'title': 'A',
                      'type': 'integer'},
                'b': {'description': 'The second number to multiply',
                      'title': 'B',
                      'type': 'integer'}},
 'required': ['a', 'b'],
 'title': 'MultiplyInput',
 'type': 'object'}


#### Toolkits

In [ ]:
from langchain.tools import tool

# create some related tools
@tool
def multiply(a: int, b: int) -> int:
    """
    Multiply two integers
    """
    return a * b

@tool
def add(a: int, b: int) -> int:
    """
    Add two integers
    """
    return a + b

In [66]:
class MathToolkit:
    def get_tools(self):
        return [multiply, add]

math_toolkit = MathToolkit()

tools = math_toolkit.get_tools()

pprint(tools)

[StructuredTool(name='multiply', description='Multiply two integers', args_schema=<class 'langchain_core.utils.pydantic.multiply'>, func=<function multiply at 0x119a34680>),
 StructuredTool(name='add', description='Add two integers', args_schema=<class 'langchain_core.utils.pydantic.add'>, func=<function add at 0x119a34220>)]


### Tool Calling in Langchain

#### Define the tool

In [9]:
@tool
def multiply(a: int, b: int) -> int:
    """
    Multiply two integers
    """
    return a * b
    
multiply.invoke({"a": 2, "b": 3})

6

In [10]:
model = ChatGroq(
    model="openai/gpt-oss-20b",
    temperature=0,
    max_tokens=None,
    reasoning_format="parsed",
    timeout=None,
    max_retries=2,
)

#### Bind the tool with the model

In [11]:
# bind the tools to the model
model_with_tools = model.bind_tools([multiply])

In [12]:
model_with_tools.invoke("hey whats your name") # here no tool call is necessary

AIMessage(content='Hey! I’m ChatGPT, your friendly AI assistant. How can I help you today?', additional_kwargs={'reasoning_content': 'User: "hey whats your name". We should respond with name. We can say "I\'m ChatGPT". No tool needed.'}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 124, 'total_tokens': 179, 'completion_time': 0.055419608, 'prompt_time': 0.006035482, 'queue_time': 0.042837598, 'total_time': 0.06145509}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_80501ff3a1', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--9e33bb61-9878-42b8-a229-841934ccfc08-0', usage_metadata={'input_tokens': 124, 'output_tokens': 55, 'total_tokens': 179})

#### Tool Calling

In [13]:
res = model_with_tools.invoke("can u multiply -34 and 32")

In [14]:
res.tool_calls # list of tool calls

[{'name': 'multiply',
  'args': {'a': -34, 'b': 32},
  'id': 'fc_38568951-f582-44ae-a5f1-3075ff743717',
  'type': 'tool_call'}]

> Note that the LLM does not run the tool only suggests which tool should be called and with what arguments

- Tool execution is handled by Langchain or by us

#### Tool execultion

In [15]:
multiply.invoke(res.tool_calls[0]['args'])

-1088

#### ToolMessage

If we directly call the tool function with the arguments we get back the result

But note what happens when u send the entire tool call dict

In [16]:
multiply.invoke(res.tool_calls[0]) # returns a ToolMessage

ToolMessage(content='-1088', name='multiply', tool_call_id='fc_38568951-f582-44ae-a5f1-3075ff743717')

We can use this `ToolMessage` as part of our chat history with the LLM

#### Using ToolMessage in a complete flow

In [17]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

In [18]:
messages = []
user_query = "can u multiply -34 and 32 and give the result in a short sentence"
user_query_message = HumanMessage(content=user_query)
messages.append(user_query_message)

result = model_with_tools.invoke(messages)
messages.append(result)

In [19]:
pprint(messages)

[HumanMessage(content='can u multiply -34 and 32 and give the result in a short sentence', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'reasoning_content': 'We need to use the multiply function. Provide a short sentence with result. Use function.', 'tool_calls': [{'id': 'fc_9e76d4b5-5095-4c8b-9258-b8722708d499', 'function': {'arguments': '{"a":-34,"b":32}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 136, 'total_tokens': 182, 'completion_time': 0.045285786, 'prompt_time': 0.006588939, 'queue_time': 0.042476551, 'total_time': 0.051874725}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_c5a89987dc', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--1c9aff63-1b2d-4031-9099-41a28189c529-0', tool_calls=[{'name': 'multiply', 'args': {'a': -34, 'b': 32}, 'id': 'fc_9e76d4b5-5095-4c8b-9258-b8722708d499', 'type': 'tool_call'}

In [22]:
result.tool_calls

[{'name': 'multiply',
  'args': {'a': -34, 'b': 32},
  'id': 'fc_9e76d4b5-5095-4c8b-9258-b8722708d499',
  'type': 'tool_call'}]

In [21]:
tool_result = multiply.invoke(result.tool_calls[0])
messages.append(tool_result)

In [23]:
messages

[HumanMessage(content='can u multiply -34 and 32 and give the result in a short sentence', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'reasoning_content': 'We need to use the multiply function. Provide a short sentence with result. Use function.', 'tool_calls': [{'id': 'fc_9e76d4b5-5095-4c8b-9258-b8722708d499', 'function': {'arguments': '{"a":-34,"b":32}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 136, 'total_tokens': 182, 'completion_time': 0.045285786, 'prompt_time': 0.006588939, 'queue_time': 0.042476551, 'total_time': 0.051874725}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_c5a89987dc', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--1c9aff63-1b2d-4031-9099-41a28189c529-0', tool_calls=[{'name': 'multiply', 'args': {'a': -34, 'b': 32}, 'id': 'fc_9e76d4b5-5095-4c8b-9258-b8722708d499', 'type': 'tool_call'}

In [24]:
model_with_tools.invoke(messages).content

'The product of –34 and 32 is –1088.'

### Currency Conversion Tool

In [25]:
import requests
from langchain_core.tools import InjectedToolArg
from typing import Annotated

In [26]:
@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
    """
    Get the conversion factor between two a base currency and a target currency
    Example:
    get_conversion_factor(base_currency="USD", target_currency="INR")
    """

    url = f"https://v6.exchangerate-api.com/v6/1e59c09b2cc7e83a7e9bece7/pair/{base_currency}/{target_currency}"
    response = requests.get(url)
    data = response.json()
    return data['conversion_rate']

get_conversion_factor.invoke({"base_currency": "USD", "target_currency": "INR"})

88.7311

`InjectedToolArg` - This is to tell the LLM that `conversion_factor` is an argument that the LLM does not need to fill in this argument which calling it as a tool - the developer will inject this value

In [104]:
@tool
def convert_and_display(base_currency_value: float, conversion_factor: Annotated[float, InjectedToolArg]) -> int:
    """
    Convert a base currency value to a target currency value using the conversion factor and display the result as a custom CurrencyBot message
    """
    amount = base_currency_value * conversion_factor
    return f"Message from currency conversion bot: the amount is {amount}"


convert_and_display.invoke({"base_currency_value": 100, "conversion_factor": 75})

'Message from currency conversion bot: the amount is 7500.0'

In [28]:
gemini_flash_model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [38]:
model = ChatGroq(
    model="openai/gpt-oss-20b",
    temperature=0,
    max_tokens=None,
)

In [91]:
model_with_tools = model.bind_tools([get_conversion_factor, convert_and_display])

In [90]:
messages = [
    HumanMessage(content="Give me conversion for 1000 USD to INR as a currencybot message")
]

first_result = model_with_tools.invoke(messages)

In [92]:
first_result

AIMessage(content='', additional_kwargs={'reasoning_content': "We need to use the function get_conversion_factor to get factor USD to INR. Then use convert_and_display with base_currency_value? The convert_and_display expects base_currency_value: number. But we need to pass the amount? The function likely uses the conversion factor internally. We need to call get_conversion_factor first. Then call convert_and_display. Let's do that.", 'tool_calls': [{'id': 'fc_bd33d143-87ad-4286-bb64-620d2a199824', 'function': {'arguments': '{"base_currency":"USD","target_currency":"INR"}', 'name': 'get_conversion_factor'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 105, 'prompt_tokens': 209, 'total_tokens': 314, 'completion_time': 0.103140022, 'prompt_time': 0.0101167, 'queue_time': 0.04403916, 'total_time': 0.113256722}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_3d587a02fb', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs':

In [93]:
first_result.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'INR'},
  'id': 'fc_bd33d143-87ad-4286-bb64-620d2a199824',
  'type': 'tool_call'}]

In [94]:
messages.append(first_result)

In [95]:
# Execute the first tool
first_tool_result = get_conversion_factor.invoke(first_result.tool_calls[0])
messages.append(first_tool_result)  # Add tool result to history

In [96]:
first_tool_result

ToolMessage(content='88.7311', name='get_conversion_factor', tool_call_id='fc_bd33d143-87ad-4286-bb64-620d2a199824')

In [97]:
messages

[HumanMessage(content='Give me conversion for 1000 USD to INR as a currencybot message', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'reasoning_content': "We need to use the function get_conversion_factor to get factor USD to INR. Then use convert_and_display with base_currency_value? The convert_and_display expects base_currency_value: number. But we need to pass the amount? The function likely uses the conversion factor internally. We need to call get_conversion_factor first. Then call convert_and_display. Let's do that.", 'tool_calls': [{'id': 'fc_bd33d143-87ad-4286-bb64-620d2a199824', 'function': {'arguments': '{"base_currency":"USD","target_currency":"INR"}', 'name': 'get_conversion_factor'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 105, 'prompt_tokens': 209, 'total_tokens': 314, 'completion_time': 0.103140022, 'prompt_time': 0.0101167, 'queue_time': 0.04403916, 'total_time': 0.113256722}, 'model_name

In [98]:
# Call model again - now it will suggest the second tool
second_result = model_with_tools.invoke(messages)

In [99]:
second_result

AIMessage(content='', additional_kwargs={'reasoning_content': 'We need to convert 1000 USD to INR. The conversion factor is 88.7311. So 1000 * 88.7311 = 88731.1 INR. We need to display as a custom CurrencyBot message. There\'s a function convert_and_display that takes base_currency_value: number. Probably expects the amount in base currency? Or maybe expects the converted value? The description: "Convert a base currency value to a target currency value using the conversion factor and display the result as a custom CurrencyBot message". So we need to call convert_and_display with base_currency_value: 1000? Or maybe the converted value? The function signature: convert_and_display({base_currency_value: number}). It likely uses the conversion factor internally. We already got the factor. So we call convert_and_display with base_currency_value: 1000.', 'tool_calls': [{'id': 'fc_17e7b91f-b4c6-4762-9d3e-560a3d0e388d', 'function': {'arguments': '{"base_currency_value":1000}', 'name': 'convert_

In [100]:
second_result.tool_calls

[{'name': 'convert_and_display',
  'args': {'base_currency_value': 1000},
  'id': 'fc_17e7b91f-b4c6-4762-9d3e-560a3d0e388d',
  'type': 'tool_call'}]

Because we had `InjectedToolArg` on `conversion_factor` this arg is not set

So we need to do that ourselves

In [48]:
second_result.tool_calls[0]['args']['conversion_factor'] = first_tool_result.content

In [50]:
messages.append(second_result)

In [51]:
messages

[HumanMessage(content='Give me conversion for 1000 USD to INR as a currencybot message', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'reasoning_content': "We need to use the function get_conversion_factor to get factor USD to INR. Then use convert_and_display with base_currency_value? The convert_and_display expects base_currency_value: number. But we need to pass the amount? The function likely uses the conversion factor internally. We need to call get_conversion_factor first. Then call convert_and_display. Let's do that.", 'tool_calls': [{'id': 'fc_5a2a2b58-210d-4236-9f84-df94a7315676', 'function': {'arguments': '{"base_currency":"USD","target_currency":"INR"}', 'name': 'get_conversion_factor'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 105, 'prompt_tokens': 209, 'total_tokens': 314, 'completion_time': 0.104664413, 'prompt_time': 0.010126352, 'queue_time': 0.042770898, 'total_time': 0.114790765}, 'model_n

In [ ]:
# invoke the second tool
second_tool_result = convert_and_display.invoke(second_result.tool_calls[0])

In [53]:
second_tool_result

ToolMessage(content='Message from currency conversion bot: the amount is 88731.09999999999', name='convert_and_display', tool_call_id='fc_dc4f931f-da63-43cd-a7ea-d5fbcaf61c86')

In [54]:
messages.append(second_tool_result)

In [ ]:
messages # now we have all the required messages in the chat history to answer the user query

[HumanMessage(content='Give me conversion for 1000 USD to INR as a currencybot message', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'reasoning_content': "We need to use the function get_conversion_factor to get factor USD to INR. Then use convert_and_display with base_currency_value? The convert_and_display expects base_currency_value: number. But we need to pass the amount? The function likely uses the conversion factor internally. We need to call get_conversion_factor first. Then call convert_and_display. Let's do that.", 'tool_calls': [{'id': 'fc_5a2a2b58-210d-4236-9f84-df94a7315676', 'function': {'arguments': '{"base_currency":"USD","target_currency":"INR"}', 'name': 'get_conversion_factor'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 105, 'prompt_tokens': 209, 'total_tokens': 314, 'completion_time': 0.104664413, 'prompt_time': 0.010126352, 'queue_time': 0.042770898, 'total_time': 0.114790765}, 'model_n

In [56]:
result = model_with_tools.invoke(messages)
result.content

'Message from currency conversion bot: the amount is 88731.09999999999'

Lets clean up this process and make it more functional

In [106]:
model = ChatGroq(
    model="openai/gpt-oss-20b",
    temperature=0,
    max_tokens=None,
)

In [107]:
model_with_tools = model.bind_tools([get_conversion_factor, convert_and_display])

In [108]:
user_query = "can u convert 1000 USD to INR and display the result as a currencybot message"

messages = [
    HumanMessage(content=user_query)
]

latest_message = messages[-1]
conversion_factor = None

while not(latest_message.type == "ai" and len(getattr(latest_message, 'tool_calls', [])) == 0):
    
    logger.debug(f"latest_message: {latest_message}")

    if latest_message.type == "human":
        latest_message = model_with_tools.invoke(messages)
        messages.append(latest_message)
    elif latest_message.type == "ai":
        tool_calls = latest_message.tool_calls
        for tool_call in tool_calls:
            if tool_call['name'] == "get_conversion_factor":
                logger.debug(f"tool_call: {tool_call}")
                tool_result = get_conversion_factor.invoke(tool_call)
                logger.debug(f"get_conversion_factor_result: {tool_result}")
                conversion_factor = tool_result.content
                messages.append(tool_result)
            elif tool_call['name'] == "convert_and_display":
                logger.debug(f"tool_call: {tool_call}")
                tool_call['args']['conversion_factor'] = conversion_factor
                tool_result = convert_and_display.invoke(tool_call)
                messages.append(tool_result)

    elif latest_message.type == "tool":
        latest_message = model_with_tools.invoke(messages)
        messages.append(latest_message)

    print (messages)
    latest_message = messages[-1]

2025-10-02 13:03:33.096 | DEBUG    | __main__:<module>:12 - latest_message: content='can u convert 1000 USD to INR and display the result as a currencybot message' additional_kwargs={} response_metadata={}
2025-10-02 13:03:33.643 | DEBUG    | __main__:<module>:12 - latest_message: content='' additional_kwargs={'reasoning_content': 'We need to use the function get_conversion_factor to get conversion factor between USD and INR. Then use convert_and_display with base_currency_value? Wait convert_and_display expects base_currency_value: number. But we need to pass the converted value? The description: "Convert a base currency value to a target currency value using the conversion factor and display the result as a custom CurrencyBot message". So convert_and_display likely uses the conversion factor internally? But we need to provide base_currency_value. It might use the conversion factor from earlier? The function likely uses the conversion factor from get_conversion_factor. But we need to 

[HumanMessage(content='can u convert 1000 USD to INR and display the result as a currencybot message', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'reasoning_content': 'We need to use the function get_conversion_factor to get conversion factor between USD and INR. Then use convert_and_display with base_currency_value? Wait convert_and_display expects base_currency_value: number. But we need to pass the converted value? The description: "Convert a base currency value to a target currency value using the conversion factor and display the result as a custom CurrencyBot message". So convert_and_display likely uses the conversion factor internally? But we need to provide base_currency_value. It might use the conversion factor from earlier? The function likely uses the conversion factor from get_conversion_factor. But we need to call get_conversion_factor first. Then call convert_and_display with base_currency_value=1000? Or maybe we need to pass the

2025-10-02 13:03:34.288 | DEBUG    | __main__:<module>:23 - get_conversion_factor_result: content='88.7311' name='get_conversion_factor' tool_call_id='fc_c91c4e5b-5445-4e85-b0bf-5c0d864bd0e7'
2025-10-02 13:03:34.289 | DEBUG    | __main__:<module>:12 - latest_message: content='88.7311' name='get_conversion_factor' tool_call_id='fc_c91c4e5b-5445-4e85-b0bf-5c0d864bd0e7'
2025-10-02 13:03:34.472 | DEBUG    | __main__:<module>:12 - latest_message: content='' additional_kwargs={'reasoning_content': 'We have conversion factor 88.7311. Need to convert 1000 USD to INR. 1000 * 88.7311 = 88731.1 INR. Then use convert_and_display function.', 'tool_calls': [{'id': 'fc_960692cc-332b-48ba-b8e3-8a7c1d2ebd4c', 'function': {'arguments': '{"base_currency_value":88731.1}', 'name': 'convert_and_display'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 72, 'prompt_tokens': 252, 'total_tokens': 324, 'completion_time': 0.070586793, 'prompt_time': 0.012164905, 'queue_time': 0.0421

[HumanMessage(content='can u convert 1000 USD to INR and display the result as a currencybot message', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'reasoning_content': 'We need to use the function get_conversion_factor to get conversion factor between USD and INR. Then use convert_and_display with base_currency_value? Wait convert_and_display expects base_currency_value: number. But we need to pass the converted value? The description: "Convert a base currency value to a target currency value using the conversion factor and display the result as a custom CurrencyBot message". So convert_and_display likely uses the conversion factor internally? But we need to provide base_currency_value. It might use the conversion factor from earlier? The function likely uses the conversion factor from get_conversion_factor. But we need to call get_conversion_factor first. Then call convert_and_display with base_currency_value=1000? Or maybe we need to pass the

2025-10-02 13:03:34.905 | DEBUG    | __main__:<module>:12 - latest_message: content='' additional_kwargs={'reasoning_content': 'We need to check the logic. The user asked: "can u convert 1000 USD to INR and display the result as a currencybot message". We called get_conversion_factor with USD to INR. It returned 88.7311. Then we called convert_and_display with base_currency_value: 88731.1. Wait, base_currency_value should be the amount in base currency? The function convert_and_display expects base_currency_value: number. But the example says: convert_and_display(base_currency_value: number). The function likely expects the base currency value to convert. But we passed 88731.1, which is the result of 1000 * 88.7311. That seems wrong. The function likely expects the base currency value (1000) and will internally multiply by conversion factor. But we passed the converted amount. So the function returned 7873208.10721, which is 88731.1 * 88.7311? Let\'s check: 88731.1 * 88.7311 = 7873208.

[HumanMessage(content='can u convert 1000 USD to INR and display the result as a currencybot message', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'reasoning_content': 'We need to use the function get_conversion_factor to get conversion factor between USD and INR. Then use convert_and_display with base_currency_value? Wait convert_and_display expects base_currency_value: number. But we need to pass the converted value? The description: "Convert a base currency value to a target currency value using the conversion factor and display the result as a custom CurrencyBot message". So convert_and_display likely uses the conversion factor internally? But we need to provide base_currency_value. It might use the conversion factor from earlier? The function likely uses the conversion factor from get_conversion_factor. But we need to call get_conversion_factor first. Then call convert_and_display with base_currency_value=1000? Or maybe we need to pass the

In [109]:
messages

[HumanMessage(content='can u convert 1000 USD to INR and display the result as a currencybot message', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'reasoning_content': 'We need to use the function get_conversion_factor to get conversion factor between USD and INR. Then use convert_and_display with base_currency_value? Wait convert_and_display expects base_currency_value: number. But we need to pass the converted value? The description: "Convert a base currency value to a target currency value using the conversion factor and display the result as a custom CurrencyBot message". So convert_and_display likely uses the conversion factor internally? But we need to provide base_currency_value. It might use the conversion factor from earlier? The function likely uses the conversion factor from get_conversion_factor. But we need to call get_conversion_factor first. Then call convert_and_display with base_currency_value=1000? Or maybe we need to pass th

In [110]:
model_with_tools.invoke(messages).content

'Message from currency conversion bot: the amount is 88731.09999999999'